In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [3]:
def read_text_file(file_path):
  with open(file_path, 'r') as file:
    content = file.read()
    return content

# Example usage:
file_path = 'data2.txt'
content = read_text_file(file_path)


'T'

In [34]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=600,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(content))

print(f"\nTotal chunks: {len(character_split_texts)}")


Total chunks: 74


In [35]:
print(character_split_texts[10])

1

.

8

 

a

c

c

o

r

d

i

n

g

 

t

o

 

A

S

T

M

-

D

-

1

2

0

4

.

 

T

h

e

 

C

O

F

-

S

t

a

t

i

c

 

v

a

l

u

e

 

i

s

 

0

.

4

9

 

a

n

d

 

t

h

e

 

C

O

F

-

K

i

n

e

t

i

c

 

v

a

l

u

e

 

i

s

 

0

.

4

1

,

 

b

o

t

h

 

m

e

a

s

u

r

e

d

 

w

i

t

h

 

A

S

T

M

 

D

-

1

8

9

4

.

 

T

h

e

 

t

r

e

a

t

m

e

n

t

 

i

s

 

C

o

r

o

n

a

 

w

i

t

h

 

a

 

v

a

l

u

e

 

o

f

 

5

4

 

a

c

c

o

r

d

i

n

g

 

t

o

 

D

-

2

5

7

8

.

 

T

h

e

 

G

S

M

 

i


In [36]:
l = character_split_texts
s = []
for i in l:
    st = ""
    for j in i:
        if j!='\n':
            st+=j
    s.append(st)

In [46]:
x = []
for i in s:
    x.append(str(i))

In [53]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function(x[1]))

c:\Users\admin\.pyenv\pyenv-win\versions\3.9.1\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[-0.029210321605205536, -0.008136014454066753, 0.03420500084757805, 0.04029564559459686, 0.07426179200410843, 0.05922503024339676, 0.08180958777666092, 0.03713740035891533, 0.03291487693786621, -0.03143708407878876, 0.06003863736987114, -0.07254361361265182, 0.024705274030566216, -0.004340923856943846, -0.013213702477514744, 0.018410928547382355, -0.08305694907903671, -0.014857950620353222, -0.1227138414978981, 0.002326284535229206, -0.033769719302654266, 0.027208760380744934, -0.02078932709991932, 0.020019397139549255, -0.011954664252698421, -0.016626909375190735, -0.02197924815118313, -0.004876185208559036, 0.0012886286713182926, -0.0873493105173111, 0.007423541974276304, 0.11303848028182983, 0.05859524756669998, -0.01921728253364563, 0.017139427363872528, -0.07474992424249649, -0.07950093597173691, -0.058088235557079315, 0.03741072118282318, 0.05945611000061035, -0.08294063061475754, -0.0891818180680275, 0.03647368401288986, -0.029225878417491913, -0.0008361443178728223, -0.0268807

In [54]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("combined", embedding_function=embedding_function)

ids = [str(i) for i in range(len(x))]

chroma_collection.add(ids=ids, documents=x)
chroma_collection.count()

74

In [55]:
query = 'elongation'

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(document)
    print('\n')

elongation, F5 value, coefficient of friction, haze, shrinkage, and surface tension. The results are given in minimum and maximum values. For instance, the overall micron ranged from 11.7 to 12.3, te


is measured in MD and TD with ASTM D-882 standard. The elongation is also measured in MD and TD with ASTM D-882 standard. The 30-minute linear shrinkage at 150 degrees Celsius is 1.8 according to AST


ensile strength (both in machine direction (MD) and transverse direction (TD)), elongation, linear shrinkage at 150 degrees Celsius, coefficient of friction (both static and kinetic), treatment corona


c. The tensile strength is measured in Kg/sqcm using the ASTM-D-882 standard, with values of 2,112 for MD and 2,422 for TD. The elongation is measured using the same standard, with a value of 130 for


The tensile strength is measured in MD and TD with ASTM D-882 standard. The elongation is also measured in MD and TD with ASTM D-882 standard. The 30-minute linear shrinkage at 150 degrees Ce

In [56]:
from openai import AzureOpenAI
from openai import OpenAI
from dotenv import load_dotenv
client = AzureOpenAI(
    api_key='32d0ee2f0fa74a60be53d08408584273',
    api_version="2023-10-01-preview",
    azure_endpoint='https://digitalsolutionnrish.openai.azure.com/',
)

In [57]:
# #Material
# Outside Coating
# Inside Coating
# Outside Treatment
# Inside Treatment

In [58]:
prompt = """You are a helpful assistant. You will be given a material as {material} and context as {context} . Your task is to output its material,outside coating,inside coating,outside treatment,inside treatment as a dictionary strictly from context given"""

In [59]:
def output_material(material):
    query = f'For the material given as {material} what is its materail,outside coating,inside coating,outside treatment and inside treatment'
    results = chroma_collection.query(query_texts=[query], n_results=5)
    retrieved_documents = results['documents'][0]
    
    messages = [
        {
            'role':'system',
            'content':prompt
        },
        {
            'role':'user',
            'content':f'material : {material} .\n context:{retrieved_documents}'
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4-32k",
        messages=messages
        
    )
    
    return response.choices[0].message.content
    


In [61]:
print(output_material("BBO  (Polyester  Film- Corona  Treated  Outside)"))

{"material": "Polyester Film", "outside coating": "None", "inside coating": "None", "outside treatment": "Corona Treated", "inside treatment": "None"}


In [62]:
list_of_materials = ['BBO  (Polyester  Film- Corona  Treated  Outside) ','CCHOB  (Chemical Coated  Wound Outside  &  Corona  Treated Inside)','POLYBSTER (PET) FILM FOR ONE SIDE CHEMICAL TREATED GRADE','POLYESTER (PET) FILM FOR PRINTING & PACKAGING GRADE','POLYESTER (PBT) FILM FOR THERMOFORMING APPLICATION']

In [63]:
data = []
for i in list_of_materials:
    data.append(output_material(i))

In [64]:
data

['{"material": "BBO (Polyester Film)", "outside coating": "None", "inside coating": "None", "outside treatment": "Corona Treated", "inside treatment": "None"}',
 '{"material": "CCHOB", "outside coating": "Chemical Coated", "inside coating": null, "outside treatment": "Wound Outside", "inside treatment": "Corona Treated"}',
 '{"material": "Polyester (PET)", "outside coating": "None", "inside coating": "None", "outside treatment": "Chemical treated", "inside treatment": "Untreated"}',
 '{"material": "Polyester (PET)", "outside coating": "None", "inside coating": "None", "outside treatment": "Corona", "inside treatment": "None"}',
 '{"material": "POLYESTER (PBT) FILM FOR THERMOFORMING APPLICATION", "outside coating": "Not mentioned in the context", "inside coating": "Not mentioned in the context", "outside treatment": "Not mentioned in the context", "inside treatment": "Not mentioned in the context"}']

In [75]:
l1 =  data[0][1:-1].split(',')
 

In [81]:
list_of_parameters = []
for i in l1:
    print(i.split(':')[0])
    list_of_parameters.append(i.split(':')[0])

"material"
 "outside coating"
 "inside coating"
 "outside treatment"
 "inside treatment"


In [83]:
for i in range(0,len(list_of_parameters)):
    list_of_parameters[i] = list_of_parameters[i][1:-1]

In [89]:

list_of_parameters = ['material','outside coating','inside coating','outside treatment','inside treatment']

In [90]:
dictionary = {}
for i in list_of_parameters:
    dictionary[i] = []

In [91]:
dictionary

{'material': [],
 'outside coating': [],
 'inside coating': [],
 'outside treatment': [],
 'inside treatment': []}

In [95]:
for i in data:
    print('))')
    l1 = i[1:-1].split(',')
    dictionary['material'].append(l1[0].split(':')[1])
    dictionary['outside coating'].append(l1[1].split(':')[1])
    dictionary['inside coating'].append(l1[2].split(':')[1])
    dictionary['outside treatment'].append(l1[3].split(':')[1])
    dictionary['inside treatment'].append(l1[4].split(':')[1])
    
    # for j in range(0,len(l1)):
        
    #     print(j.split(':')[1])

))
))
))
))
))


In [96]:
dictionary

{'material': [' "BBO (Polyester Film)"',
  ' "CCHOB"',
  ' "Polyester (PET)"',
  ' "Polyester (PET)"',
  ' "POLYESTER (PBT) FILM FOR THERMOFORMING APPLICATION"'],
 'outside coating': [' "None"',
  ' "Chemical Coated"',
  ' "None"',
  ' "None"',
  ' "Not mentioned in the context"'],
 'inside coating': [' "None"',
  ' null',
  ' "None"',
  ' "None"',
  ' "Not mentioned in the context"'],
 'outside treatment': [' "Corona Treated"',
  ' "Wound Outside"',
  ' "Chemical treated"',
  ' "Corona"',
  ' "Not mentioned in the context"'],
 'inside treatment': [' "None"',
  ' "Corona Treated"',
  ' "Untreated"',
  ' "None"',
  ' "Not mentioned in the context"']}

In [97]:
import pandas as pd


In [98]:
df = pd.DataFrame(dictionary)

In [99]:
df

,material,outside coating,inside coating,outside treatment,inside treatment
0,"""BBO (Polyester Film)""","""None""","""None""","""Corona Treated""","""None"""
1,"""CCHOB""","""Chemical Coated""",null,"""Wound Outside""","""Corona Treated"""
2,"""Polyester (PET)""","""None""","""None""","""Chemical treated""","""Untreated"""
3,"""Polyester (PET)""","""None""","""None""","""Corona""","""None"""
4,"""POLYESTER (PBT) FILM FOR THERMOFORMING APPLI...","""Not mentioned in the context""","""Not mentioned in the context""","""Not mentioned in the context""","""Not mentioned in the context"""


In [100]:
df.to_csv('data.csv')